<a href="https://colab.research.google.com/github/ventodeco/genetic_algorithm/blob/master/sistem_cerdas_genetic_algorithm_with_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def inisiasi_populasi():
  # data harga saham mandiri Oktober akhir hingga april
  harga_saham = [6425, 6400, 6425, 6675, 6625, 6775, 6800, 6525, 6550, 6650,
                 6725, 6500, 6425, 6425, 6475, 6550, 6600, 6600, 6575, 6150,
                 6275, 6375, 6325, 6375, 6375, 6250, 6350, 6475, 6500, 6500,
                 6500, 6500, 6575, 6550, 6575, 6500, 6475, 6700, 6575, 7050,
                 7300, 7375, 7300, 7200, 7250, 7375, 6975, 6925, 6725, 6850,
                 6925, 6950, 6850, 6550, 6475, 6425, 6425, 6500, 6325, 6425,
                 6525, 6350, 6400, 6700, 6700, 6875, 6775, 6700, 6725, 6775,
                 6800, 6675, 6700, 6600, 6575, 6650, 6525, 6325, 6650, 6725,
                 6525, 6425, 6400, 6300, 6325, 6350, 6250, 6200, 6250, 6175,
                 6375, 6375, 6150, 6050, 5975, 5700, 6000, 5975, 6525, 6425]
  
  # memasvenukkan harga saham ke dalam individu masing masing
  jumlah_individu = 10
  list_individu = []
  iterasi_saham = 0

  for iterasi in range(jumlah_individu):
    ind = []
    for it in range(jumlah_individu):
      ind.append(harga_saham[iterasi_saham])
      iterasi_saham += 1
    list_individu.append(ind)

  # cari random konstanta
  list_konstanta = []
  min_num = -5
  max_num = 5

  for iterasi in range(jumlah_individu):
    row = []
    for it in range(jumlah_individu):
      row.append(randint(min_num, max_num))
    list_konstanta.append(row)

  # memecah menjadi dua populasi, 
  # setiap populasi memiliki 5 individu

  # list_konstanta
  # list_individu

  populasi_individu = []
  populasi_konstanta = []
  jumlah_populasi = 2
  isi_tiap_populasi = 5
  iterasi_individu = 0

  for iterasi in range(jumlah_populasi):
    row_individu = []
    row_konstanta = []
    for it in range(isi_tiap_populasi):
      row_konstanta.append(list_konstanta[iterasi_individu])
      row_individu.append(list_individu[iterasi_individu])
      iterasi_individu += 1
    populasi_individu.append(row_individu)
    populasi_konstanta.append(row_konstanta)

  return populasi_konstanta, populasi_individu

In [2]:
def fitness(graded):
  # inisiasi target untuk f(x)
  target_harga_saham = 7000
  return [ abs(harga-target_harga_saham)  for harga in graded ]

In [3]:
def grading_populasi(individu, konstanta):
  pop1 = []
  pop2 = []
  # f(x) = a1 * x1 + .. + a10 * x10
  for x in range(5):
    pop1.append(reduce(add, [num1 * num2 for num1, num2 in zip(individu[0][x], konstanta[0][x])]))
    pop2.append(reduce(add, [num1 * num2 for num1, num2 in zip(individu[1][x], konstanta[1][x])]))

  return pop1, pop2

In [4]:
def ambil_parent(graded, list_simpan, list_konstanta, next_parent):
  for x in range(2):
    if list_simpan[graded[x][0]][0] > graded[x][1]:
      list_simpan[graded[x][0]][0] = graded[x][1]
      list_simpan[graded[x][0]][1] = list_konstanta[graded[x][0]]
    next_parent.append(list_konstanta[graded[x][0]])

  return list_simpan, next_parent

In [5]:
def crossover_populasi(old_parent):
  new_parent = []
  new_parent.append(old_parent[0])
  new_parent.append(old_parent[1])
  new_parent.append(old_parent[0][:5] + old_parent[1][5:])
  new_parent.append(old_parent[0][:3] + old_parent[1][3:6] + old_parent[0][6:])
  new_parent.append(old_parent[1][:3] + old_parent[0][3:6] + old_parent[1][6:])

  return new_parent

In [6]:
def mutasi_populasi(new_parent):
  min_num = -5
  max_num = 5
  chance_to_change = 0.5
  for x in range(5):
    if random() > chance_to_change:
      new_parent[x][randint(0, 9)] = randint(min_num, max_num)
      new_parent[x][randint(0, 9)] = randint(min_num, max_num)
  return new_parent

In [45]:
from random import randint
from operator import add
from functools import reduce
from random import random

import pprint
pp = pprint.PrettyPrinter(indent=2)

BIG_INT = 999999999

harga_saham = [6425, 6400, 6425, 6675, 6625, 6775, 6800, 6525, 6550, 6650,
                 6725, 6500, 6425, 6425, 6475, 6550, 6600, 6600, 6575, 6150,
                 6275, 6375, 6325, 6375, 6375, 6250, 6350, 6475, 6500, 6500,
                 6500, 6500, 6575, 6550, 6575, 6500, 6475, 6700, 6575, 7050,
                 7300, 7375, 7300, 7200, 7250, 7375, 6975, 6925, 6725, 6850,
                 6925, 6950, 6850, 6550, 6475, 6425, 6425, 6500, 6325, 6425,
                 6525, 6350, 6400, 6700, 6700, 6875, 6775, 6700, 6725, 6775,
                 6800, 6675, 6700, 6600, 6575, 6650, 6525, 6325, 6650, 6725,
                 6525, 6425, 6400, 6300, 6325, 6350, 6250, 6200, 6250, 6175,
                 6375, 6375, 6150, 6050, 5975, 5700, 6000, 5975, 6525, 6425]

# ---------------------------------------------------
# dapat populasi davenri konstotanta dan inddeividu dimcoana
# ada satu list ada 2 populasi dan setiap populasi 
# terdapat 5 individu

# inisiasi parent awal
list_parent1 = [ [BIG_INT, 0] for x in range(5) ]
list_parent2 = [ [BIG_INT, 0] for x in range(5) ]

populasi_konstanta, populasi_individu = inisiasi_populasi()

# mulai dari generasi ke 1
gen = 1
while True:
  print('GENERASI {}'.format(gen))

  print('populasi individu')
  pp.pprint(populasi_individu)

  print('populasi konstanta')
  pp.pprint(populasi_konstanta)
  # grading dengan rumus mencari f(x)
  graded_populasi1, graded_populasi2 = grading_populasi(populasi_individu, populasi_konstanta)

  print('Graded Populasi')
  print(graded_populasi1)
  print(graded_populasi2)

  # get fitness
  fitness_pop1 = fitness(graded_populasi1)
  fitness_pop2 = fitness(graded_populasi2)
  print('fitness')
  print(fitness_pop1)
  print(fitness_pop2)

  # sort dengan index original
  sorted_fitness1 = sorted(enumerate(fitness_pop1), key=lambda i: i[1])
  sorted_fitness2 = sorted(enumerate(fitness_pop2), key=lambda i: i[1])

  print('sorted fitness untuk ambil parent terbaik')
  print(sorted_fitness1)
  print(sorted_fitness2)

  # list new parent
  new_parent1 = []
  new_parent2 = []

  list_parent1, new_parent1 = ambil_parent(sorted_fitness1, list_parent1, populasi_konstanta[0], new_parent1)
  list_parent2, new_parent2 = ambil_parent(sorted_fitness2, list_parent2, populasi_konstanta[1], new_parent2)

  print('parent_new')
  pp.pprint(new_parent1)
  pp.pprint(new_parent2)
  co_parent1 = crossover_populasi(new_parent1)
  co_parent2 = crossover_populasi(new_parent2)

  print('hasil crossover')
  pp.pprint(co_parent1)
  pp.pprint(co_parent2)

  parent_mutate1 = mutasi_populasi(co_parent1)
  parent_mutate2 = mutasi_populasi(co_parent2)
  print('mutasi')
  pp.pprint(parent_mutate1)
  pp.pprint(parent_mutate2)
  populasi_konstanta = []

  populasi_konstanta.append(parent_mutate1)
  populasi_konstanta.append(parent_mutate2)
  print('konstanta akhir')
  pp.pprint(populasi_konstanta)
  # print('oi')

  pp.pprint(list_parent1)
  pp.pprint(list_parent2)


  # penghentian evolusi
  flag_parent1 = True
  flag_parent2 = True
  for x in range(5):
    if BIG_INT == list_parent1[x][0]:
      flag_parent1 = False
    if BIG_INT == list_parent2[x][0]:
      flag_parent2 = False
    
  if flag_parent1 and flag_parent2:
    break

  gen += 1

print('generasi berhenti pada gen ke {} dengan nilai saham dengan list_konstanta : '.format(gen))
pp.pprint(list_parent1)
pp.pprint(list_parent2)

GENERASI 1
populasi individu
[ [ [6425, 6400, 6425, 6675, 6625, 6775, 6800, 6525, 6550, 6650],
    [6725, 6500, 6425, 6425, 6475, 6550, 6600, 6600, 6575, 6150],
    [6275, 6375, 6325, 6375, 6375, 6250, 6350, 6475, 6500, 6500],
    [6500, 6500, 6575, 6550, 6575, 6500, 6475, 6700, 6575, 7050],
    [7300, 7375, 7300, 7200, 7250, 7375, 6975, 6925, 6725, 6850]],
  [ [6925, 6950, 6850, 6550, 6475, 6425, 6425, 6500, 6325, 6425],
    [6525, 6350, 6400, 6700, 6700, 6875, 6775, 6700, 6725, 6775],
    [6800, 6675, 6700, 6600, 6575, 6650, 6525, 6325, 6650, 6725],
    [6525, 6425, 6400, 6300, 6325, 6350, 6250, 6200, 6250, 6175],
    [6375, 6375, 6150, 6050, 5975, 5700, 6000, 5975, 6525, 6425]]]
populasi konstanta
[ [ [-5, -2, 3, -5, -3, 0, 0, -4, -1, -2],
    [2, -5, -5, -2, 5, 1, 1, 4, -2, -1],
    [-2, 4, 4, -4, -4, 0, -1, 1, -1, 2],
    [-5, 2, 3, -4, -5, 3, 4, -4, -4, 4],
    [4, -5, 0, 1, 4, 1, 1, -5, 2, 2]],
  [ [-3, -3, 3, -2, -1, -4, -1, -3, -2, -5],
    [0, 0, 4, -3, -3, 2, 1, -3, -4, 3],


In [58]:
print('Prediksi harga saham 50 hari kedepan')

index_prediksi = []

# pp.pprint(list_parent1[0][1])
# pp.pprint(harga_saham)

for iterasi in range(5):
  index_prediksi.append(list_parent1[iterasi][1])

for iterasi in range(5):
  index_prediksi.append(list_parent2[iterasi][1])

# pp.pprint(index_prediksi)

jumlah_individu = 10
list_individu = []
iterasi_saham = 0

for iterasi in range(jumlah_individu):
  ind = []
  for it in range(jumlah_individu):
    ind.append(harga_saham[iterasi_saham])
    iterasi_saham += 1
  list_individu.append(ind)
prediksi_harga = []
# f(x) = a1 * x1 + .. + a10 * x10
default_harga = 7000
iterasi = 0
for x in range(50):
  if iterasi >= 10:
    iterasi = 0
  harga = reduce(add, [num1 * num2 for num1, num2 in zip(list_individu[iterasi], index_prediksi[iterasi])])
  if (harga - default_harga) > 20000:
    harga -= 10000
  
  harga = harga - randint(100, 300)

  prediksi_harga.append(abs(harga))
  iterasi += 1

# pp.pprint(prediksi_harga)

hari = 1
for x in range(50):
  print('Prediksi harga saham hari ke-{} adalah Rp {},00'.format(hari, prediksi_harga[x]))
  hari += 1


Prediksi harga saham 50 hari kedepan
Prediksi harga saham hari ke-1 adalah Rp 5399,00
Prediksi harga saham hari ke-2 adalah Rp 6018,00
Prediksi harga saham hari ke-3 adalah Rp 35068,00
Prediksi harga saham hari ke-4 adalah Rp 6215,00
Prediksi harga saham hari ke-5 adalah Rp 8921,00
Prediksi harga saham hari ke-6 adalah Rp 8805,00
Prediksi harga saham hari ke-7 adalah Rp 21630,00
Prediksi harga saham hari ke-8 adalah Rp 22129,00
Prediksi harga saham hari ke-9 adalah Rp 7807,00
Prediksi harga saham hari ke-10 adalah Rp 2046,00
Prediksi harga saham hari ke-11 adalah Rp 5343,00
Prediksi harga saham hari ke-12 adalah Rp 6047,00
Prediksi harga saham hari ke-13 adalah Rp 35085,00
Prediksi harga saham hari ke-14 adalah Rp 6154,00
Prediksi harga saham hari ke-15 adalah Rp 8811,00
Prediksi harga saham hari ke-16 adalah Rp 8843,00
Prediksi harga saham hari ke-17 adalah Rp 21569,00
Prediksi harga saham hari ke-18 adalah Rp 22169,00
Prediksi harga saham hari ke-19 adalah Rp 7670,00
Prediksi harga s